In [ ]:
# Mout Google Drive
# https://towardsdatascience.com/google-drive-google-colab-github-dont-just-read-do-it-5554d5824228
from google.colab import drive
ROOT = "/content/drive"
drive.mount(ROOT)
# %pwd %ls
# run github settings
%run /content/drive/MyDrive/CNNStanford/pytorch/pytorch_sandbox/Colab_Helper.ipynb

In [ ]:
MESSAGE = "clean file & gitignore again"
!git config --global user.email "ronyginosar@mail.huji.ac.il"
!git config --global user.name "ronyginosar"
!git add .

In [ ]:
!git commit -m "{MESSAGE}"
!git push "{GIT_PATH}"

Empty

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
from torchvision import transforms
from pathlib import Path
import matplotlib.pyplot as plt

CIFAR_ROOT = "" #add cifar path

# This function is copied from http://www.cs.toronto.edu/~kriz/cifar.html
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

# (1) Implement CIFAR10 as a Custom PyTorch Dataset
class CIFAR10(Dataset):
    def __init__(self, split, root_dir=CIFAR_ROOT):
        assert split in ["train", "test"]
        # Implement this function
        ...

    def __len__(self):
        # Implement this function
        ...

    def __getitem__(self, idx):
        # Implement this function
        ...

# (2) Create Dataset objects of the above class, for both 'train' and 'test' splits
cifar_train = ...
cifar_test = ...

# (3) Verify by inspecting and displaying a single sample from the dataset


# (4) Create DataLoaders, using the datasets from (2), for both 'train' and 'test' splits.
# Use batch size 16.
# Verify by generating an example batch and inspect its shape, data type and data range.
cifar_train_loader = ...
cifar_test_loader = ...



# (5) Repeat steps (2) to (4) Using Torchvision's built-in CIFAR10 dataset
# See https://pytorch.org/vision/stable/datasets.html#torchvision.datasets.CIFAR10
# Get a sample of the dataset. What data type is it?
# Make sure that the sample's image data is a float tensor between 0 and 1,
# By setting the 'transform' input argument. (Hint - ToTensor)

cifar_train_tv = datasets.CIFAR10(... ,
                                  root=, download=True)
cifar_test_tv = datasets.CIFAR10(... ,
                                  root=, download=True)

cifar_train_loader_tv = ...
cifar_test_loader_tv = ...




# (6) Verify that you get the same data from both datasets (yours and torchvision)
# Here I wrote the sciprt for you.
# Go over it and make sure that you get 'Passed' on all 3 checks.

data0, label0 = cifar_train[19483]
data1, label1 = cifar_train_tv[19483]
if torch.equal(data0, data1) and label0==label1:
    print('Dataset Comparison: Passed!')
else:
    print('DatasetComparison: Failed!')

test_batch_data0, test_batch_label0 = next(iter(cifar_test_loader))
test_batch_data1, test_batch_label1 = next(iter(cifar_test_loader_tv))
if torch.equal(test_batch_data0, test_batch_data1) \
        and torch.equal(test_batch_label0, test_batch_label1):
    print('Test Dataloader Comparison: Passed!')
else:
    print('Test Dataloader Comparison: Failed!')

# Q: Why do we need to set the same manual seed before iterating through each dataloader?
torch.manual_seed(0)
train_batch_data0, train_batch_label0 = next(iter(cifar_train_loader))
torch.manual_seed(0)
train_batch_data1, train_batch_label1 = next(iter(cifar_train_loader_tv))
if torch.equal(train_batch_data0, train_batch_data1) \
        and torch.equal(train_batch_label0, train_batch_label1):
    print('Train Dataloader Comparison: Passed!')
else:
    print('Train Dataloader Comparison: Failed!')


# (7) BONUS: Add multiple data augmentations to the train dataset
# for example:
# - random flipping
# - random color jitter
# - random grayscale
# Test the augmentations by visualizing a few examples
# Use the built-in torchvision transforms.
# See  https://pytorch.org/vision/stable/transforms.html

Solution

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
from torchvision import transforms
from pathlib import Path
import matplotlib.pyplot as plt

CIFAR_ROOT = ""# #add cifar path

# see http://www.cs.toronto.edu/~kriz/cifar.html
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

# (1) Implement CIFAR10 as a Custom PyTorch Dataset
class CIFAR10(Dataset):
    def __init__(self,split, root_dir=CIFAR_ROOT, transform=None):
        assert split in ["train", "test"]
        if split=="train":
            batch_files = sorted(Path(root_dir).glob("data_batch*"))
        elif split=="test":
            batch_files = sorted(Path(root_dir).glob("test_batch*"))

        for i,file in enumerate(batch_files):
            dict = unpickle(file)
            if i==0:
                data = torch.tensor(dict[b'data'])
                labels = torch.tensor(dict[b'labels'])
            else:
                data = torch.cat((data,torch.tensor(dict[b'data'])))
                labels = torch.cat((labels,torch.tensor(dict[b'labels'])))

        classes = unpickle(Path(root_dir) / "batches.meta")
        classes = [str(l, "utf-8") for l in classes[b'label_names']]

        self.data = data.view(-1, 3, 32, 32)
        self.labels = labels
        self.classes = classes
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image = self.data[idx].to(dtype=torch.float)/255
        if self.transform is not None:
            image = self.transform(image)
        label = self.labels[idx].item()
        return image, label

    def show(self,idx):
        img, lbl = self[idx]
        plt.figure()
        plt.imshow(img.permute([1,2,0]))
        plt.title(f'label={self.classes[lbl]}')
        plt.show()

def main(argv):

    # (2) Create Dataset objects of the above class, for both 'train' and 'test' splits
    cifar_train = CIFAR10("train")
    cifar_test = CIFAR10("test")

    # (3) Verify by displaying a single sample from the dataset
    cifar_train.show(34645)

    # (4) Create DataLoaders, using the datasets from (2), for both 'train' and 'test' splits.
    # Use batch size 16.
    # Verify by generating an example batch and inspect its shape, data type and data range.
    cifar_train_loader = DataLoader(cifar_train,batch_size=16, shuffle=True)
    train_batch_data, train_batch_lbl = next(iter(cifar_train_loader))

    cifar_test_loader = DataLoader(cifar_test,batch_size=16)
    test_batch_data, test_batch_lbl = next(iter(cifar_train_loader))



    # (5) Repeat steps (2) to (4) Using Torchvision's built-in CIFAR10 dataset
    cifar_train_tv = datasets.CIFAR10(train=True, transform=transforms.ToTensor(),
                                      root="", download=True)#add cifar path
    cifar_test_tv = datasets.CIFAR10(train=False, transform=transforms.ToTensor(),
                                      root="", download=True)#add cifar path
    cifar_train.show(34645)
    cifar_train_loader_tv = DataLoader(cifar_train_tv, batch_size=16, shuffle=True)
    cifar_test_loader_tv = DataLoader(cifar_test_tv,batch_size=16)


    # (6) Verify that you get the same data from both datasets (yours and torchvision)
    data0, label0 = cifar_train[19483]
    data1, label1 = cifar_train_tv[19483]
    if torch.equal(data0, data1) and label0==label1:
        print('Dataset Comparison: Passed!')
    else:
        print('DatasetComparison: Failed!')

    test_batch_data0, test_batch_label0 = next(iter(cifar_test_loader))
    test_batch_data1, test_batch_label1 = next(iter(cifar_test_loader_tv))
    if torch.equal(test_batch_data0, test_batch_data1) \
            and torch.equal(test_batch_label0, test_batch_label1):
        print('Test Dataloader Comparison: Passed!')
    else:
        print('Test Dataloader Comparison: Failed!')

    torch.manual_seed(0)
    train_batch_data0, train_batch_label0 = next(iter(cifar_train_loader))
    torch.manual_seed(0)
    train_batch_data1, train_batch_label1 = next(iter(cifar_train_loader_tv))
    if torch.equal(train_batch_data0, train_batch_data1) \
            and torch.equal(train_batch_label0, train_batch_label1):
        print('Train Dataloader Comparison: Passed!')
    else:
        print('Train Dataloader Comparison: Failed!')

if __name__ == "__main__":
    main(sys.argv)